In [3]:
!pip install transformers

In [4]:
!pip install sentencepiece

In [5]:
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer, GPT2LMHeadModel, GPT2Tokenizer

In [6]:
from transformers import T5Tokenizer

In [7]:
import json
import torch
from transformers import Trainer, TrainingArguments

In [8]:
!pip install datasets

In [9]:
from datasets import Dataset

In [10]:
import torch, os, re, pandas as pd, json
from sklearn.model_selection import train_test_split
from transformers import DataCollatorForLanguageModeling, DataCollatorWithPadding, GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments, AutoConfig

In [11]:
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
# the eos and bos tokens are defined
bos = '<|endoftext|>'
eos = '<|EOS|>'
pad = '<|pad|>'

special_tokens_dict = {'eos_token': eos, 'bos_token': bos, 'pad_token': pad}

# the new token is added to the tokenizer
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)

# the model config to which we add the special tokens
config = AutoConfig.from_pretrained('gpt2',
                                    bos_token_id=tokenizer.bos_token_id,
                                    eos_token_id=tokenizer.eos_token_id,
                                    pad_token_id=tokenizer.pad_token_id,
                                    output_hidden_states=False)
model = GPT2LMHeadModel.from_pretrained(model_name, config=config)
# the model embedding is resized
model.resize_token_embeddings(len(tokenizer))

Embedding(50259, 768)

In [12]:
!pip install --quiet pytorch-lightning

In [13]:
import pandas as pd
import numpy as np
import torch
from pathlib import Path
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl
from sklearn.model_selection import train_test_split

# from transformers import (
#     T5ForConditionalGeneration,
#     T5TokenizerFast as T5Tokenizer
# )

from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger

import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm

In [14]:
pl.seed_everything(42)

INFO:lightning_fabric.utilities.seed:Seed set to 42


42

In [15]:
# Load your JSON dataset
with open('/content/Train_Headline_Generation.json', 'r') as json_file:
    #train_dataset = json.load(json_file)
    train_dataset=pd.read_json(json_file)

# train_dataset=pd.read_json('/content/Train_Headline_Generation.json')

In [16]:
# Load your dev JSON dataset
with open('/content/Dev_Headline_Generation.json', 'r') as json_file:
    # dev_dataset = json.load(json_file)
    dev_dataset=pd.read_json(json_file)

In [17]:
train_dataset["news"].str.len().max(), train_dataset["headline"].str.len().max()

(3023, 127)

In [18]:
train_dataset.head()

,news,headline
0,"(Oct 7, 2014 12:40 PM CDT) As of Jan. 1, Walm...",30K Walmart Part-Timers to Lose Health Insurance
1,"(Oct 29, 2013 8:15 AM CDT) Dax Shepard and Kr...",Dax Shepard: Wedding to Kristen Bell Cost $142
2,"(Mar 6, 2016 10:50 AM) Nancy Reagan, the help...",Nancy Reagan Dead at 94
3,"(Aug 15, 2008 5:11 AM CDT) American Airlines ...",American Airlines Faces $7M Fine for Safety Vi...
4,"(Apr 18, 2016 1:02 PM CDT) Ingrid Lyne, the S...",$222K Raised for Kids of Mom Dismembered on Date


In [19]:
dev_dataset.head()

,news,headline
0,"(Oct 5, 2011 12:11 PM CDT) Police are still h...",3rd Victim Dead in Quarry Shooting; Manhunt St...
1,"(Nov 4, 2008 3:19 PM) Stocks rallied on Elect...",Stocks Up 305 in Election Rally
2,"(Dec 24, 2014 11:19 AM) Turns out you won't e...",You Can Watch The Interview at 1pm
3,"(Oct 16, 2014 3:02 AM CDT) Tristen Kurilla, t...","Murder Suspect, 10, Will Stay in Adult Jail"
4,"(Oct 3, 2011 9:10 AM CDT) Tobacco companies w...",Tobacco Firms Knew of Radiation in Cigs—in 1959


In [20]:
x=dev_dataset.iloc[0]
x["news"]

'(Oct 5, 2011  12:11 PM CDT) Police are still hunting for an  armed and dangerous  man who shot and killed three people and wounded five others at the California quarry where he worked this morning. He also shot and injured another woman in an attempted carjacking later. The suspect is identified as 45-year-old Shareef Allman. The San Jose Mercury News says he moonlighted as producer of a show called Real 2 Real on something called CreaTV in the area. (He interviews Jesse Jackson in one YouTube clip, which is in the gallery. Allman, who spread a message of non-violence, also has written a book called Amazing Grace about female victims of domestic violence, notes the Mercury News. The shooting spree occurred about 4:30am local time when a man left a safety meeting at the Lehigh Southwest Cement Permanente Plant, then returned with a rifle and a handgun, according to police. He fled on foot, and a massive manhunt is under way in the San Jose area. Schools are on lockdown, notes AP.'

In [21]:
import re

def remove_timestamp_from_article(article):
    # Define a regular expression pattern to match the timestamp at the beginning
    timestamp_pattern = r'^\([^)]+\)\s*'

    # Use re.sub to replace the timestamp with an empty string
    cleaned_article = re.sub(timestamp_pattern, '', article)

    return cleaned_article

In [22]:
# Assuming train_dataset and dev_dataset are your DataFrames
train_dataset["news"] = train_dataset["news"].apply(remove_timestamp_from_article)
dev_dataset["news"] = dev_dataset["news"].apply(remove_timestamp_from_article)

In [23]:
train_dataset.head()

,news,headline
0,"As of Jan. 1, Walmart will no longer offer 30,...",30K Walmart Part-Timers to Lose Health Insurance
1,Dax Shepard and Kristen Bell got married at th...,Dax Shepard: Wedding to Kristen Bell Cost $142
2,"Nancy Reagan, the helpmate, backstage adviser,...",Nancy Reagan Dead at 94
3,American Airlines faces FAA fines of more than...,American Airlines Faces $7M Fine for Safety Vi...
4,"Ingrid Lyne, the Seattle mom allegedly murdere...",$222K Raised for Kids of Mom Dismembered on Date


In [24]:
dev_dataset.head()

,news,headline
0,Police are still hunting for an armed and dan...,3rd Victim Dead in Quarry Shooting; Manhunt St...
1,Stocks rallied on Election Day as investors ap...,Stocks Up 305 in Election Rally
2,Turns out you won't even have to leave your ho...,You Can Watch The Interview at 1pm
3,"Tristen Kurilla, the Pennsylvania 10-year-old ...","Murder Suspect, 10, Will Stay in Adult Jail"
4,Tobacco companies were well aware that their p...,Tobacco Firms Knew of Radiation in Cigs—in 1959


In [25]:
print(len(train_dataset))
print(len(dev_dataset))

21157
2365


In [26]:
train_dataset["article"]= bos + ' ' + train_dataset['news'] + ' ' + 'TL;DR:' + ' ' + train_dataset['headline'] + ' ' + eos
dev_dataset["article"]= bos + ' ' + dev_dataset['news'] + ' ' + 'TL;DR:' + ' ' + dev_dataset['headline'] + ' ' + eos

In [27]:
import torch, os, re, pandas as pd, json
from sklearn.model_selection import train_test_split
from transformers import DataCollatorForLanguageModeling, DataCollatorWithPadding, GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments, AutoConfig
from datasets import Dataset

In [28]:
# we load the datasets directly from a pandas df
train_dataset_article = Dataset.from_pandas(train_dataset[['article']])
dev_dataset_article = Dataset.from_pandas(dev_dataset[['article']])

In [29]:
train_dataset_article

Dataset({
    features: ['article'],
    num_rows: 21157
})

In [30]:
def tokenize_function(examples):
  return tokenizer(examples['article'], padding=True)


tokenized_train_dataset = train_dataset_article.map(
    tokenize_function,
    batched=True,
    num_proc=5,
    remove_columns=['article'],
)
tokenized_val_dataset = dev_dataset_article.map(
    tokenize_function,
    batched=True,
    num_proc=5,
    remove_columns=['article'],
)

Map (num_proc=5):   0%|          | 0/21157 [00:00<?, ? examples/s]

Map (num_proc=5):   0%|          | 0/2365 [00:00<?, ? examples/s]

In [31]:
! pip install accelerate -U

In [32]:
! pip install transformers[torch]

In [33]:
model_headlines_path = './model_headlines_news'

training_args = TrainingArguments(
    output_dir=model_headlines_path,          # output directory
    num_train_epochs=1,              # total # of training epochs
    per_device_train_batch_size=4,  # batch size per device during training
    per_device_eval_batch_size=4,   # batch size for evaluation
    warmup_steps=200,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir=model_headlines_path,            # directory for storing logs
    prediction_loss_only=True,
    save_steps=10000
)

In [34]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [35]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset
)
trainer.train()

Step,Training Loss
500,9.555500
1000,3.146500
1500,3.123600
2000,3.099400
2500,3.073800
3000,3.044200
3500,3.042700
4000,3.041300
4500,3.034400
5000,3.026600


TrainOutput(global_step=5290, training_loss=3.6808939077453036, metrics={'train_runtime': 3313.1922, 'train_samples_per_second': 6.386, 'train_steps_per_second': 1.597, 'total_flos': 5618198184192000.0, 'train_loss': 3.6808939077453036, 'epoch': 1.0})

In [36]:
trainer.save_model()
tokenizer.save_pretrained(model_headlines_path)

('./model_headlines_news/tokenizer_config.json',
 './model_headlines_news/special_tokens_map.json',
 './model_headlines_news/vocab.json',
 './model_headlines_news/merges.txt',
 './model_headlines_news/added_tokens.json')

In [37]:
def generate_n_text_samples(model, tokenizer, input_text):
  text_ids = tokenizer.encode(input_text, return_tensors = 'pt')
  # text_ids = text_ids.to(device)
  # model = model.to(device)

  generated_text_samples = model.generate(
      text_ids,
      max_length= 150,
      no_repeat_ngram_size= 2,
      repetition_penalty= 1.5,
      # top_p= 0.92,
      # temperature= .85,
      #do_sample= True,
      #top_k= 125,
      length_penalty=1.0,
      early_stopping= True
  )
  # gen_text = []
  # for t in generated_text_samples:
  #     text = tokenizer.decode(t, skip_special_tokens=True)
  #     gen_text.append(text)

  #     return gen_text
  preds = [
    tokenizer.decode(gen_id, skip_special_tokens=True, clean_up_tokenization_spaces=True)
    for gen_id in generated_text_samples
  ]

  return "".join(preds)

# def generate_headline(text):
#   text_encoding = tokenizer(
#       text,
#       max_length=512,
#       padding="max_length",
#       truncation=True,
#       return_attention_mask=True,
#       add_special_tokens=True,
#       return_tensors="pt"
#   )

#   generated_ids = model.model.generate(
#       input_ids=text_encoding["input_ids"],
#       attention_mask=text_encoding["attention_mask"],
#       max_length=150,
#       num_beams=2, # beam search
#       repetition_penalty=2.5,
#       length_penalty=1.0,
#       early_stopping=True # To speed up the process
#   )

#   preds = [
#       tokenizer.decode(gen_id, skip_special_tokens=True, clean_up_tokenization_spaces=True)
#       for gen_id in generated_ids
#   ]

#   return "".join(preds)

In [40]:
# trained model loading
model_headlines_path = './model_headlines_news'


headlines_model = GPT2LMHeadModel.from_pretrained(model_headlines_path)
headlines_tokenizer = GPT2Tokenizer.from_pretrained(model_headlines_path)

# Checking if CUDA (GPU) is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

input_text = dev_dataset_article['article'][0]

headlines = generate_n_text_samples(headlines_model, headlines_tokenizer,
                                    input_text)
print(headlines.split("TL;DR:")[1])

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:418: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1281: UserWarning: Input length of input_ids is 229, but

 3rd Victim Dead in Quarry Shooting; Manhunt Still On  shooter


In [43]:
dev_dataset['headline'][0]

'3rd Victim Dead in Quarry Shooting; Manhunt Still On'

In [44]:
dev_dataset['headline'][1]

'Stocks Up 305 in Election Rally'

In [53]:
headlines = generate_n_text_samples(headlines_model, headlines_tokenizer,
                                    dev_dataset_article['article'][1])
print(headlines.split("TL;DR:")[1])

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1281: UserWarning: Input length of input_ids is 185, but `max_length` is set to 150. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(


 Stocks Up 305 in Election Rally  Market


In [54]:
dev_dataset['headline'][10]

'Deficit Breaks $1T Again'

In [55]:
dev_dataset['article'][10]

'<|endoftext|> The federal budget deficit is set to break a trillion dollars for the second year in a row, the Obama administration is set to report today. Officials foresee a deficit of $1.29 trillion for the 2010 budget year that ended Sept. 30, the AP reports, down from last year’s $1.4 trillion. Next year’s could soar back to that record high, the administration projects. TL;DR: Deficit Breaks $1T Again <|EOS|>'

In [56]:
dev_dataset_article['article'][10]

'<|endoftext|> The federal budget deficit is set to break a trillion dollars for the second year in a row, the Obama administration is set to report today. Officials foresee a deficit of $1.29 trillion for the 2010 budget year that ended Sept. 30, the AP reports, down from last year’s $1.4 trillion. Next year’s could soar back to that record high, the administration projects. TL;DR: Deficit Breaks $1T Again <|EOS|>'

In [57]:
headlines = generate_n_text_samples(headlines_model, headlines_tokenizer,
                                    dev_dataset_article['article'][10])
print(headlines.split("TL;DR:")[1])

 Deficit Breaks $1T Again  Budget Forecast by White House  deficits will be at their lowest level since 2008 and they are expected not only over time but also as long-term projections—and it's likely more than double what was projected earlier this month when President Bush announced his plan


In [58]:
!pip install rouge

In [59]:
from rouge import Rouge
ROUGE = Rouge()

In [60]:
res = {}
res['rouge-1'] = {}
res['rouge-2'] = {}
res['rouge-l'] = {}
for key in res:
  res[key]['r'] = 0
  res[key]['p'] = 0
  res[key]['f'] = 0
for index in range(len(dev_dataset.index)):
  sample_row = dev_dataset.iloc[index]
  # text = sample_row["news"]
  reference = sample_row["headline"]
  candidate = generate_n_text_samples(headlines_model, headlines_tokenizer, dev_dataset_article['article'][index])
  score = ROUGE.get_scores(candidate.split("TL;DR:")[1], reference)
  for key in res:
    res[key]['r'] += score[0][key]['r']
    res[key]['p'] += score[0][key]['p']
    res[key]['f'] += score[0][key]['f']
for key in res:
  res[key]['r'] /= len(dev_dataset.index)
  res[key]['p'] /= len(dev_dataset.index)
  res[key]['f'] /= len(dev_dataset.index)
res

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:418: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1281: UserWarning: Input length of input_ids is 229, but `max_length` is set to 150. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1281: UserWarning: Input length of input_ids is 185, but `max_length` is set to 150. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1281: UserWarning: Input length of input_ids is 367, but `max_length` is set to 150. This can lead 

{'rouge-1': {'r': 0.9979900179688762,
  'p': 0.8101442750667496,
  'f': 0.8735974031089239},
 'rouge-2': {'r': 0.9976138461022183,
  'p': 0.7918384569781086,
  'f': 0.8607797027756273},
 'rouge-l': {'r': 0.9979900179688762,
  'p': 0.8101442750667496,
  'f': 0.8735974031089239}}